# 本地调用tool工具-使用AppBuilderEventHandler简化tool_call操作

基于multi_tool_call的示例，我们可以使用Client应用来执行tool_call操作，完成指定的命令，但是需要自己配置client的思考与运行流程，较为繁琐。SDK提供了使用AppBuilderEventHandler简化tool_call操作的功能,这里我们将调用Client应用，使其调用Text2Image工具生成图片，并将文件下载到本地

##### 配置运行环境&导入Client应用

In [2]:
import os
import appbuilder


# AppBuilder Token，替换为您个人的Token
os.environ["APPBUILDER_TOKEN"] = "your api key"

# 应用为：智能问题解决者
app_id = "e97865e7-e1be-45d3-ab8a-ea84ca6e0b9a"
app_client = appbuilder.AppBuilderClient(app_id)
conversation_id = app_client.create_conversation()

##### 继承AppBuilderEventHandler类实现一个Agent框架的定义

In [3]:
from appbuilder.core.console.appbuilder_client.event_handler import AppBuilderEventHandler
class MyEventHandler(AppBuilderEventHandler):
    def execute_local_command(self, cmd: str):
        import subprocess
        try:
            result = subprocess.check_output(cmd, shell=True).decode("utf-8")
            if result.strip() == "":
                return "命令执行成功，无返回值"
            return result
        except Exception as e:
            return str(e)
    
    def interrupt(self, run_context, run_response):
        thought = run_context.current_thought
        # 绿色打印
        print("\033[1;32m", "-> Agent 中间思考: ", thought, "\033[0m")

        tool_output = []
        for tool_call in run_context.current_tool_calls:
            tool_call_id = tool_call.id
            tool_res = self.execute_local_command(
                **tool_call.function.arguments)
            # 蓝色打印
            print("\033[1;34m", "-> 本地ToolCall结果: \n", tool_res, "\033[0m\n")
            tool_output.append(
                {
                    "tool_call_id": tool_call_id,
                    "output": tool_res
                }
            )
        return tool_output
    
    def success(self, run_context, run_response):
        print("\n\033[1;31m","-> Agent 非流式回答: \n", run_response.answer, "\033[0m")

    def running(self, run_context, run_response):
        print("\n\033[1;31m","-> Agent 流式回答: \n", run_response.answer, "\033[0m")

##### 定义本地的tools工具

In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "execute_local_command",
            "description": "可以在bash环境中，执行输入的指令, 注意，一次只能执行一个完整的原子命令。并且下载文件使用curl命令，因为没有安装wget，例如：ls",
            "parameters": {
                "type": "object",
                "properties": {
                    "cmd": {
                        "type": "string",
                        "description": "需要执行的指令",
                    },
                },
                "required": ["cmd"],
            },
        },
    }
]

##### 运行Client实现实现链路的调用

In [ ]:
with app_client.run_with_handler(
        conversation_id = conversation_id,
        query = "请为我生成一张男生头像照片，并下载到本地",
        tools = tools,
        event_handler = MyEventHandler(),
    ) as run:
        run.until_done()

- Agent运行流程及输出
  
![Agent运行流程及输出](https://bj.bcebos.com/v1/appbuilder-sdk-components/Agent%E8%BF%90%E8%A1%8C%E6%B5%81%E7%A8%8B.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-08-22T08%3A38%3A58Z%2F-1%2Fhost%2F25e5b0ac328376a048c919d16c8d6fab9044d09ea2bb4b8b7614b958bb40182f)

- 查看下载至本地的图片

![载至本地的图片](https://bj.bcebos.com/v1/appbuilder-sdk-components/boy_avatar.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-08-22T08%3A40%3A26Z%2F-1%2Fhost%2F5d6f2f482bc72b487f59f7bee18e68aacf0399b71210e11a16e99f59754c64c6)